In [18]:
from langchain import hub

from langchain.schema import SystemMessage


mp_prompt = hub.pull("hwchase17/react-multi-input-json")

In [20]:
mp_prompt.messages.append(SystemMessage(content="You are a data-aware agent that has access to computational and experimental materials data on Materials Project."))

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format

In [27]:
mp_prompt.messages[0].prompt.template = "You are a helpful agent having access to materials data on Materials Project." + mp_prompt.messages[0].prompt.template

'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'

In [44]:


from mp_api.client import MPRester

import dotenv

dotenv.load_dotenv()

with MPRester() as mpr:

    mpr.materials.elasticity.search()

    elastic_docs = mpr.materials.elasticity._search(
        **{"material_ids": "mp-149", "fields": ["material_id", "formula_pretty", "elastic_tensor"]}
        # query={"material_id": "mp-149"},
        # fields=["elasticity"]
    )
    print(mpr.materials.elasticity.available_fields)

    print(elastic_docs[0])
    

    robodocs = mpr.materials.robocrys._search(
        material_ids="mp-149,mp-19017,mp-1002",
        # fields=["description"]
    )

    print(len(robodocs))

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'order', 'elastic_tensor', 'compliance_tensor', 'bulk_modulus', 'shear_modulus', 'sound_velocity', 'thermal_conductivity', 'young_modulus', 'universal_anisotropy', 'homogeneous_poisson', 'debye_temperature', 'fitting_data', 'fitting_method', 'state']
MPDataDoc<ElasticityDoc>
formula_pretty='Si',
material_id=MPID(mp-149),
elastic_tensor=ElasticTensorDoc(raw=((153.19279267761982, 56.777185976088205, 56.777185976088205, -7.979727989493311e-17, -6.661338147750937e-16, 9.436895709313829e-16), (56.777185976088205, 153.19279267761982, 56.77718597608822, -6.661338147750937e-16, -7.979727989493311e-17, 6.839865079851815e-15), (56.777185976088205, 56.77718597608822, 153.19279267761985, -6.

Retrieving RobocrystallogapherDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

2


In [13]:
robodocs[0]

MPDataDoc<RobocrystallogapherDoc>(
builder_meta=EmmetMeta(emmet_version='0.65.0', pymatgen_version='2023.7.17', pull_request=990, database_version='2023.11.1', build_date=datetime.datetime(2023, 8, 27, 2, 21, 17, 245000), license=None),
nsites=None,
elements=None,
nelements=None,
composition=None,
composition_reduced=None,
formula_pretty=None,
formula_anonymous=None,
chemsys=None,
volume=None,
density=None,
density_atomic=None,
symmetry=None,
property_name='robocrys',
material_id=MPID(mp-19017),
deprecated=False,
deprecation_reasons=None,
last_updated=datetime.datetime(2023, 3, 11, 11, 38, 26, 705000),
origins=[],
warnings=[],
description='LiFePO₄ is Hausmannite-derived structured and crystallizes in the orthorhombic Pnma space group. Li¹⁺ is bonded to six O²⁻ atoms to form LiO₆ octahedra that share corners with four equivalent FeO₆ octahedra, corners with two equivalent PO₄ tetrahedra, edges with two equivalent LiO₆ octahedra, edges with two equivalent FeO₆ octahedra, and edges with t